In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict, RepeatedKFold
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier, \
RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import itertools as it
import time as time
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

import warnings
from IPython import display
import re

from xgboost import XGBClassifier, XGBRegressor


In [13]:
train = pd.read_csv('train_classification.csv')
test = pd.read_csv('test_classification.csv')
train.head()
# train.columns
h_id = test.host_id
test.head()

id     host_id  host_since host_location  \
0  1543972437713169913  1521551465  2020-12-16           NaN   
1  1710552057351883447   429737865  2016-12-16  New York, NY   
2             97075525   995041819  2019-03-14           NaN   
3             83734823    97552339  2014-12-02   Chicago, IL   
4             56722823  1029857633  2019-04-22           NaN   

  host_response_time host_response_rate host_acceptance_rate  \
0       within a day                88%                  80%   
1     within an hour               100%                  97%   
2     within an hour                99%                  99%   
3     within an hour               100%                 100%   
4       within a day               100%                  65%   

    host_neighbourhood  host_listings_count  host_total_listings_count  \
0       Near West Side                   44                         44   
1            Cambridge                 4820                       6438   
2  West Loop/Greektown                   71                         88   
3        Streeterville                   14                         47   
4          River North                   20                         22   

                 host_verifications host_has_profile_pic  \
0                ['email', 'phone']                    t   
1  ['email', 'phone', 'work_email']                    t   
2                         ['phone']                    t   
3                ['email', 'phone']                    t   
4                ['email', 'phone']                    t   

  host_identity_verified neighbourhood_cleansed   latitude  longitude  \
0                      t           Lincoln Park  41.931102 -87.667468   
1                      t        Near North Side  41.891162 -87.622263   
2                      t        Near North Side  41.895350 -87.626040   
3                      t        Near North Side  41.891376 -87.620269   
4                      t        Near North Side  41.894760 -87.625640   

        property_type        room_type  accommodates bathrooms_text  beds  \
0  Entire rental unit  Entire home/apt            14         1 bath   1.0   
1  Entire rental unit  Entire home/apt             2         1 bath   1.0   
2  Entire rental unit  Entire home/apt             8        2 baths   3.0   
3  Entire rental unit  Entire home/apt             4         1 bath   1.0   
4        Entire condo  Entire home/apt             3         1 bath   1.0   

   minimum_nights  maximum_nights  minimum_minimum_nights  \
0              32             730                      32   
1              32            1125                      32   
2               3             344                       2   
3              32            1125                      32   
4              32             365                      32   

   maximum_minimum_nights  minimum_maximum_nights  maximum_maximum_nights  \
0                      32                     730                     730   
1                     366                    1125                    1125   
2                       3                    1125                    1125   
3                      32                    1125                    1125   
4                      32                     365                     365   

   minimum_nights_avg_ntm  maximum_nights_avg_ntm has_availability  \
0                    32.0                   730.0                t   
1                   354.2                  1125.0                t   
2                     2.2                  1125.0                t   
3                    32.0                  1125.0                t   
4                    32.0                   365.0                t   

   availability_30  availability_60  availability_90  availability_365  \
0               30               60               90               365   
1                0                1               31               306   
2               20               47               77               34

In [14]:
pd.set_option('display.max_columns', None)

# train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)

train['host_response_rate'] = train['host_response_rate'].str.rstrip('%').astype(float)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float)
train['host_since'] = pd.to_datetime(train['host_since'])

train['host_is_superhost'] = train['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

train['instant_bookable'] = train['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
train['host_has_profile_pic'] = train['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
train['host_identity_verified'] = train['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
train['has_availability'] = train['has_availability'].apply(lambda x: 1 if x == 't' else 0)


'------------------------------------------------------------------------------------------------'

test['host_response_rate'] = test['host_response_rate'].str.rstrip('%').astype(float)
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.rstrip('%').astype(float)
test['host_since'] = pd.to_datetime(test['host_since'])

test['instant_bookable'] = test['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
test['host_has_profile_pic'] = test['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
test['host_identity_verified'] = test['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
test['has_availability'] = test['has_availability'].apply(lambda x: 1 if x == 't' else 0)


# test['host_is_superhost'] = test['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)


# covert host_since into a usuable form
current_date = pd.to_datetime("today")

train['days_since_host_started'] = (current_date - train['host_since']).dt.days
test['days_since_host_started'] = (current_date - test['host_since']).dt.days

# Impute missing values

#Impute numerical columns with the mean
for column in train.select_dtypes(include=['number']).columns:
    train[column].fillna(train[column].mean(), inplace=True)
    
for column in test.select_dtypes(include=['number']).columns:
    test[column].fillna(train[column].mean(), inplace=True)  # Use train's mean to avoid data leakage

# Impute categorical columns with the mode (most frequent value)
for column in train.select_dtypes(include=['object', 'category']).columns:
    train[column].fillna(train[column].mode()[0], inplace=True)
    
for column in test.select_dtypes(include=['object', 'category']).columns:
    test[column].fillna(train[column].mode()[0], inplace=True)  # Use train's mode to avoid data leakage


# print(test.isnull().any())
# print('---------------------------------------------------------')
# print(train.isnull().any())

def extract_bathrooms(text):
    if 'half' in text.lower():
        return 0.5
    else:
        match = re.search(r'\d+\.?\d*', text)
        return float(match.group()) if match else np.nan

train['bathrooms'] = train['bathrooms_text'].apply(extract_bathrooms)
test['bathrooms'] = test['bathrooms_text'].apply(extract_bathrooms)

train['bathrooms'].fillna(train['bathrooms'].mean(), inplace=True)
test['bathrooms'].fillna(test['bathrooms'].mean(), inplace=True)


train_categories = set(train['host_location'].unique())
test['host_location'] = test['host_location'].apply(lambda x: x if x in train_categories else np.nan)
test['host_location'].fillna(train['host_location'].mode()[0], inplace=True)

train['host_verifications_count'] = train['host_verifications'].apply(lambda x: len(x.split(',')))
test['host_verifications_count'] = test['host_verifications'].apply(lambda x: len(x.split(',')))

train.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)
test.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)

train = pd.get_dummies(train)
test = pd.get_dummies(test)




train.head()

id  host_response_rate  host_acceptance_rate  \
0            107745711                96.0                  95.0   
1             78539203               100.0                  97.0   
2  1112961466060884803               100.0                 100.0   
3            104794437               100.0                  98.0   
4  1491898404543717797               100.0                 100.0   

   host_is_superhost  host_listings_count  host_total_listings_count  \
0                  0                   11                         13   
1                  0                 1141                       2346   
2                  0                    1                          1   
3                  0                   55                         58   
4                  1                   74                         75   

   host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                     1                       1  41.794240 -87.661380   
1                     1                       1  41.892749 -87.620711   
2                     1                       1  41.948160 -87.652309   
3                     1                       1  41.801030 -87.597160   
4                     1                       1  41.855770 -87.624130   

   accommodates  beds  minimum_nights  maximum_nights  minimum_minimum_nights  \
0             1   1.0              32            1125                      32   
1            12   3.0              32             365                      32   
2             6   3.0               2              45                       2   
3             2   1.0               2             180                       2   
4             6   3.0               2             365                       2   

   maximum_minimum_nights  minimum_maximum_nights  maximum_maximum_nights  \
0                      32                    1125                    1125   
1                      32                     365                     365   
2                       2                      45                      45   
3                       2                     180                     180   
4                       2                    1125                    1125   

   minimum_nights_avg_ntm  maximum_nights_avg_ntm  has_availability  \
0                    32.0                  1125.0                 1   
1                    32.0                   365.0                 1   
2                     2.0                    45.0                 1   
3                     2.0                   180.0                 1   
4                     2.0                  1125.0                 1   

   availability_30  availability_60  availability_90  availability_365  \
0               30               60               90               365   
1               25               49               66               135   
2                0                0                0                 0   
3               23               53               83               355   
4                0                0                0                 0   

   number_of_reviews  number_of_reviews_ltm  number_of_reviews_l30d  \
0                 18                      0                       0   
1                  0                      0                       0   
2                 14                     14                       1   
3                 13                     13                       3   
4                 64                     42                       3   

   review_scores_rating  review_scores_accuracy  review_scores_cleanliness  \
0              4.940000                5.000000                   4.890000   
1              4.760427                4.794096                   4.748194   
2              4.870000                5.000000                   5.000000   
3              4.080000                4.080000                   4.000000   
4              4.800000                4.740000                   4.750000   

   review_scores_chec

In [15]:
train_cols = list(train.columns)
test_cols = list(test.columns)

train_cols = [col for col in train_cols if col in test_cols]
test_cols = [col for col in test_cols if col in train_cols]

train_cols == test_cols

True

In [16]:
x_train = train[train_cols].drop(columns=['id'])
y_train = train.host_is_superhost

x_test = test[test_cols].drop(columns=['id'])

# Remove restricted characters
x_train.columns = x_train.columns.str.replace('[', '', regex=True).str.replace(']', '', regex=True).str.replace('<', '', regex=True)
x_test.columns = x_test.columns.str.replace('[', '', regex=True).str.replace(']', '', regex=True).str.replace('<', '', regex=True)

In [17]:
pos = y_train.value_counts()[0] / y_train.value_counts()[1]

In [21]:
model1 = XGBClassifier(random_state = 1, 
                     scale_pos_weight = pos,
                     n_estimators=50, 
                     max_depth=6,
                     subsample=0.75,
                     learning_rate=0.1,
                    
                     reg_lambda = 1,
                     gamma = 0,
                    )
model1.fit(x_train, y_train)

y_pred = model1.predict(x_test)

In [22]:
feature_importances = model1.feature_importances_

features_importance = pd.DataFrame({'feature': x_train.columns, 'importance': feature_importances})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

features_importance

feature  importance
26               review_scores_rating    0.097105
24              number_of_reviews_ltm    0.048022
30        review_scores_communication    0.032820
28          review_scores_cleanliness    0.028215
5              host_identity_verified    0.027377
..                                ...         ...
158  host_neighbourhood_Garfield Park    0.000000
157       host_neighbourhood_Galewood    0.000000
156      host_neighbourhood_Gage Park    0.000000
155  host_neighbourhood_Fulton Market    0.000000
380             room_type_Shared room    0.000000

[381 rows x 2 columns]

In [24]:
important_features = list(features_importance.head(20).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

model = XGBClassifier(random_state = 1, 
                     scale_pos_weight = pos,
                     n_estimators=50, 
                     max_depth=6,
                     subsample=0.75,
                     learning_rate=0.1,
                    
                     reg_lambda = 1,
                     gamma = 0,
                    )
model.fit(x_train_reduced, y_train)

y_pred = model.predict(x_test_reduced)

In [28]:
# coarse grid

model = XGBClassifier(random_state=1, scale_pos_weight = pos)

grid = {
    'n_estimators' : [100, 1000],
    'max_depth' : [4,8],
    'learning_rate' : [0.01, 0.1, 1],
    'subsample' : [0.5, 0.75, 1.0],
    'reg_lambda':[0.01, 0.1, 1],
    'gamma':[0, 1] 
    
}

gscv = GridSearchCV(model, grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 1000, 'reg_lambda': 0.1, 'subsample': 0.75}
0.8776371308016877


In [30]:
# finer grid

model = XGBClassifier(random_state=1, scale_pos_weight = pos)

grid = {
    'n_estimators' : [500, 1000],
    'max_depth' : [6,8],
    'learning_rate' : [0.001, 0.01, 0.02],
    'subsample' : [0.6, 0.75, 0.85],
    'reg_lambda':[0.01, 0.1, 1],
    'gamma':[0, 0.1] 
    
}

gscv = GridSearchCV(model, grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
{'gamma': 0.1, 'learning_rate': 0.02, 'max_depth': 6, 'n_estimators': 1000, 'reg_lambda': 0.01, 'subsample': 0.85}
0.8816556158328309


In [32]:
# finest grid

model = XGBClassifier(random_state=1, scale_pos_weight = pos)

grid = {
    'n_estimators' : [750, 1000, 2000, 5000],
    'max_depth' : [6,8],
    'learning_rate' : [0.015, 0.02, 0.05],
    'subsample' : [0.85, 0.95, 1.0],
    'reg_lambda':[0.01, 0.02, 0.05],
    'gamma':[0.01, 0.1, 0.2] 
    
}

gscv = GridSearchCV(model, grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 3 folds for each of 648 candidates, totalling 1944 fits
{'gamma': 0.01, 'learning_rate': 0.015, 'max_depth': 6, 'n_estimators': 1000, 'reg_lambda': 0.02, 'subsample': 1.0}
0.8852722523608599


In [36]:
# tuning threshold

best_model = gscv.best_estimator_

probas = cross_val_predict(best_model, x_train_reduced, y_train, cv=3, method='predict_proba')

thrs = np.arange(0.0, 1.001, 0.001)
best_accuracy = 0
best_thr = 0.5

for thr in thrs:
    preds = (probas[:, 1] >= thr).astype(int)
    accuracy = accuracy_score(y_train, preds)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_thr = thr

print(f"Best Threshold: {best_thr} with Accuracy: {best_accuracy}")

probs = best_model.predict_proba(x_test_reduced)[:,1]

y_pred = (probs >= best_thr).astype(int)

Best Threshold: 0.504 with Accuracy: 0.8860759493670886


In [38]:
# shuffle = true

cv_settings = StratifiedKFold(random_state=1, n_splits=5, shuffle=True)

model = XGBClassifier(random_state=1, scale_pos_weight = pos)

grid = {
    'n_estimators' : [100, 1000],
    'max_depth' : [4,8],
    'learning_rate' : [0.01, 0.1, 1],
    'subsample' : [0.5, 0.75, 1.0],
    'reg_lambda':[0.01, 0.1, 1],
    'gamma':[0, 1] 
    
}

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict_proba(x_test_reduced)[:,1]

y_pred = (probs >= best_thr).astype(int)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 1000, 'reg_lambda': 0.1, 'subsample': 1.0}
0.8860747512663721


In [40]:
# repeated kfold

cv_settings = RepeatedStratifiedKFold(random_state=1, n_repeats=5, n_splits=5)

model = XGBClassifier(random_state=1, scale_pos_weight = pos)

grid = {
    'n_estimators' : [1000, 2000, 5000],
    'max_depth' : [6,8],
    'learning_rate' : [0.01, 0.1, 0.2, 0.5],
    'subsample' : [1.0],
    'reg_lambda':[0.01, 0.1, 0.2, 0.5],
    'gamma':[0] 
    
}

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict_proba(x_test_reduced)[:,1]

y_pred = (probs >= best_thr).astype(int)

Fitting 25 folds for each of 96 candidates, totalling 2400 fits
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 2000, 'reg_lambda': 0.5, 'subsample': 1.0}
0.8869201428830901


In [42]:
model = model1

perm_importance = permutation_importance(model, x_train, y_train, n_repeats=5, random_state=1)

feature_names = [col for col in x_train if col != 'id' and col != 'host_is_superhost']
features_importance = pd.DataFrame({'feature': feature_names, 'importance': perm_importance.importances_mean})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

features_importance

feature  importance
2                       host_listings_count    0.105767
24                    number_of_reviews_ltm    0.077838
26                     review_scores_rating    0.047941
39                  days_since_host_started    0.016677
0                        host_response_rate    0.015712
..                                      ...         ...
380                   room_type_Shared room    0.000000
304     neighbourhood_cleansed_Logan Square   -0.000040
251  host_neighbourhood_West Loop/Greektown   -0.000040
51                host_location_Chicago, IL   -0.000080
310  neighbourhood_cleansed_Near North Side   -0.000080

[381 rows x 2 columns]

In [43]:
important_features = list(features_importance.head(15).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

model = XGBClassifier(random_state=1, scale_pos_weight = pos)

grid = {
    'n_estimators' : [100, 1000],
    'max_depth' : [4,8],
    'learning_rate' : [0.01, 0.1, 1],
    'subsample' : [0.5, 0.75, 1.0],
    'reg_lambda':[0.01, 0.1, 1],
    'gamma':[0, 1] 
    
}

gscv = GridSearchCV(model, grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 1000, 'reg_lambda': 0.1, 'subsample': 0.75}
0.8949166164356037


In [44]:
best_model = gscv.best_estimator_

probas = cross_val_predict(best_model, x_train_reduced, y_train, cv=5, method='predict_proba')

thrs = np.arange(0.0, 1.001, 0.001)
best_accuracy = 0
best_thr = 0.5

for thr in thrs:
    preds = (probas[:, 1] >= thr).astype(int)
    accuracy = accuracy_score(y_train, preds)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_thr = thr

print(f"Best Threshold: {best_thr} with Accuracy: {best_accuracy}")

probs = best_model.predict_proba(x_test_reduced)[:,1]

y_pred = (probs >= best_thr).astype(int)

Best Threshold: 0.507 with Accuracy: 0.8973277074542897


In [45]:
ids = test.id
results = pd.DataFrame({
    'id': ids,
    'predicted': y_pred
})


results.to_csv('XGboost_Classifier_submission.csv', index=False)
results.head(30)

id  predicted
0   1543972437713169913          0
1   1710552057351883447          0
2              97075525          1
3              83734823          1
4              56722823          0
5             103823601          1
6   1159028243437550345          0
7              92485765          0
8              11142401          1
9   1189875082289533291          0
10  1544820103115167001          1
11             26331127          0
12             58383089          1
13             64762469          0
14            100902043          0
15            103130147          0
16             94352581          0
17  1849015146225985937          0
18             91307779          1
19             53591385          0
20             84071033          0
21             99484949          0
22             97864023          0
23             31809543          0
24  1519081708190436577          1
25             59771473          1
26  1310888620900225001          0
27             23054201          1
28             37470529          0
29  1212015341587401973          1